# L5: Content Creation at Scale

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

## Initial Imports

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Creating Structured Output

In [2]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")

## Loading Tasks and Agents YAML files

In [3]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Importing CrewAI Tools

In [4]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

## Setup Multi LLM models

> Note: The 'llama-3.1-70b-versatil' model was deprecated. As a result, we updated it to the 'llama-3.3-70b-versatil' model in January 2025.

In [5]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
groq_llm = "groq/llama-3.3-70b-versatile"

## Creating Crew, Agents, and Tasks

In [6]:
# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config['market_news_monitor_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=groq_llm,
)

data_analyst_agent = Agent(
    config=agents_config['data_analyst_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
    llm=groq_llm,
)

content_creator_agent = Agent(
    config=agents_config['content_creator_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
)

quality_assurance_agent = Agent(
    config=agents_config['quality_assurance_agent'],
)

# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config['monitor_financial_news'],
    agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config['analyze_market_data'],
    agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config['create_content'],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance'],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput
)

# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
    verbose=True
)

2025-02-24 16:19:03,951 - 140241060215680 - sqlite.py-sqlite:111 - WARNING: ⚠️ It looks like you upgraded from a version below 0.6 and could benefit from vacuuming your database. Run chromadb utils vacuum --help for more information.


## Kicking off the Crew

In [7]:
result = content_creation_crew.kickoff(inputs={
  'subject': 'Inflation in the US and the impact on the stock market in 2024'
})

# Agent: Lead Market Analyst
## Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular focus on Inflation in the US and the impact on the stock market in 2024. Identify and summarize the most relevant and impactful news items that could influence market trends or investor decisions. Utilize financial news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends, regulatory changes, or significant economic indicators that directly relate to Inflation in the US and the impact on the stock market in 2024.



# Agent: Lead Market Analyst
## Thought: Thought: To create a comprehensive report on the latest news and updates related to Inflation in the US and its impact on the stock market in 2024, I need to gather information from reliable financial news sources and utilize real-time market data tools. I will start by searching the internet for relevant news articles and updates on Inflation in the US a



# Agent: Lead Market Analyst
## Thought: content
 end of page
Thought: Based on the content of the website https://www.usbank.com/investing/financial-perspectives/market-news/federal-reserve-tapering-asset-purchases.html, it appears that the Federal Reserve has been calibrating its policy to keep inflation in check. The Fed has been raising interest rates to combat high inflation and has been reducing its balance sheet of financial assets. The current federal funds target rate is 4.25% to 4.50%, and the Fed has indicated that it will continue to reduce its bond holdings.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"2024 US inflation rate and stock market impact\"}"
## Tool Output: 

Search results: Title: Analysis: Assessing Inflation's Impact - U.S. Bank
Link: https://www.usbank.com/investing/financial-perspectives/investing-insights/how-does-inflation-affect-investments.html
Snippet: The inflation rate for the 12 months ending January 2024 (as measured b



# Agent: Lead Market Analyst
## Final Answer: 
Here is the final answer: 

**Detailed Summary Report: Inflation in the US and its Impact on the Stock Market in 2024**

Introduction:
The US economy has experienced significant fluctuations in recent years, with inflation rates rising to historic highs in 2022. The Federal Reserve has been working to combat this inflation by raising interest rates and reducing its balance sheet of financial assets. This report will provide an overview of the current state of inflation in the US and its impact on the stock market in 2024.

Current State of Inflation:
The annual inflation rate in the US rose for a third consecutive month to 2.9% in December 2024, in line with market expectations. This increase in inflation has been driven by a combination of factors, including rising labor costs, higher energy prices, and increased demand for goods and services.

Impact on the Stock Market:
The rising inflation rate has had a mixed impact on the stock mar



# Agent: Chief Data Strategist
## Thought: Title: Current US Inflation Rate Statistics: Jan 2024 vs Jan 2025
Link: https://inflationdata.com/Inflation/Inflation_Rate/CurrentInflationRate.asp
Snippet: Annual U.S. inflation rate: 3.0%, 0.3% change from Jan. 2024 to Jan. 2025. Monthly U.S. inflation rate: 0.5%, -0.1% change from Dec. 2024.
Thought: From the search results, I have gathered information on the current inflation rate in the US, which is 2.9% as of December 2024. The core inflation rate, excluding food and energy, is 3.2%. The inflation rate has been rising over the past few months, with a 0.2% increase in November and a 0.3% increase in December. I will use this information to analyze the impact of inflation on the stock market.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"impact of inflation on stock market 2024\", \"description\": \"Search for the impact of inflation on the stock market in 2024\"}"
## Tool Output: 

Search results: Title: Analy



# Agent: Chief Data Strategist
## Final Answer: 
**Detailed Summary Report: Inflation in the US and its Impact on the Stock Market in 2024**

Introduction:
The US economy has experienced significant fluctuations in recent years, with inflation rates rising to historic highs in 2022. The Federal Reserve has been working to combat this inflation by raising interest rates and reducing its balance sheet of financial assets. This report will provide an overview of the current state of inflation in the US and its impact on the stock market in 2024.

Current State of Inflation:
The annual inflation rate in the US rose for a third consecutive month to 2.9% in December 2024, in line with market expectations. This increase in inflation has been driven by a combination of factors, including rising labor costs, higher energy prices, and increased demand for goods and services.

Impact on the Stock Market:
The rising inflation rate has had a mixed impact on the stock market. On the one hand, high

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  5.12it/s]




# Agent: Creative Content Director
## Thought: Thought: I have gathered relevant information on the current state of inflation and its impact on the stock market for 2024. I can now create content that educates and informs the audience based on this information, focusing on key insights and predictions.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"inflation US impact on stock market 2024\", \"website\": \"https://www.jpmorgan.com/insights/outlook/economic-outlook/economic-trends\"}"
## Tool Output: 
Relevant Content:
outlook. In this scenario, continued geopolitical conflicts and trade actions cause inflation to stay higher for longer, driving the Federal Reserve to hike rates.1 Scenarios Baseline (70%): Real GDP growth slowed overall in the first quarter of 2024, coming in very close to our previous forecast’s expectations.2 We expect GDP growth will continue to moderate through the second half of this year and the start of the next, but the stor

## Social Content

In [8]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']
for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)

Twitter
📈 As we head into 2024, inflation remains a key
concern for investors. How will you adjust your
strategies? Check our insights! #Investing
#Inflation #StockMarket
--------------------------------------------------
LinkedIn
Understanding the impact of inflation on the U.S.
stock market in 2024 is crucial for informed
investment. Dive into our latest analysis to learn
more! #EconomicInsights #InvestSmart
--------------------------------------------------
Facebook
Inflation is shaping the economic landscape as we
approach 2024. Learn about strategies to navigate
this uncertainty in our new article! #Inflation
#Investing
--------------------------------------------------


## Blog Post

In [9]:
from IPython.display import display, Markdown
display(Markdown(result.pydantic.dict()['article']))

# Understanding Inflation in the U.S. and Its Impact on the Stock Market in 2024

As we approach 2024, inflation continues to be a focal point in the U.S. economy. Inflation, defined as the rate at which the general level of prices for goods and services rises, erodes purchasing power and can significantly impact investment strategies. Recent data suggests that inflation rates may stabilize in the coming year, but the effects on the stock market remain a topic of extensive analysis and debate.

The forecast for inflation in 2024 indicates a potential gradual decline, thanks to a combination of Federal Reserve policies aimed at controlling volatility and supply chain improvements. This stabilization could create a more predictable environment for investors, allowing for strategic planning and risk management. However, it's essential to remain vigilant as external factors, such as geopolitical tensions and global economic shifts, can easily influence inflation trajectories. 

## Key Insights from Economic Analysts

1. **Interest Rate Adjustments:** The Federal Reserve's approach to interest rates will play a crucial role in shaping market reactions. Analysts predict slight increases in rates could accompany high inflation periods in 2024, which may lead to a cautious but necessary reevaluation of equity investments. 

2. **Sector Impact Variability:** Different sectors may react uniquely to inflationary pressures. Historically, consumer staples tend to perform better during inflationary times, as they provide essential goods. Conversely, discretionary sectors may face challenges and slower growth. Investors should consider diversifying their portfolios to mitigate risks associated with sector-specific downturns.

## Strategies for Investors in 2024

To navigate the potential uncertainty of inflation and its effects on the stock market, investors should adopt proactive strategies:

- **Diversification:** Expanding one's investment portfolio to include a mix of assets—such as bonds, commodities, and international stocks—can help cushion against inflation impacts. Consider allocating more dollars to inflation-resistant assets, such as Real Estate Investment Trusts (REITs) and Treasury Inflation-Protected Securities (TIPS).

- **Focus on Quality:** With the potential for increased volatility, prioritizing investments in companies with strong balance sheets, consistent earnings growth, and competitive advantages is essential. Quality investments often outperform in turbulent economic conditions.

For a more detailed analysis of specific sectors impacted by inflation, refer to our article on [Sector Performance in Inflationary Markets](#).

## Concluding Thoughts

As inflation continues to reshape the economic landscape in 2024, staying informed and strategically positioned will be crucial for investors. By understanding the nuances of inflation and its effects on the stock market, making informed and timely investment decisions becomes more attainable. 

Stay ahead of the curve by continually monitoring economic indicators and adjusting your investment strategies as needed. Engaging with financial news platforms and expert insights can further empower you in navigating the complexities of inflation and market performance in the year ahead.

---

### Follow Us for Real-Time Updates
Stay connected with our latest analyses and updates by following us on:

- [Twitter](https://twitter.com/yourcompany)
- [LinkedIn](https://linkedin.com/company/yourcompany)
- [Facebook](https://facebook.com/yourcompany)

Your informed decisions can help you navigate this intricate economic environment. For more information and resources, visit our website at [yourcompany.com](https://yourcompany.com).
